In [1]:
import numpy as np
import pandas as pd
import os, sys  
import matplotlib.pyplot as plt

In [2]:
route = os.getcwd()
new_route = os.path.dirname(route)
sys.path.append(route)

In [3]:
from utils.mining_data_tb import merge_csv
from utils.mining_data_tb import save_merged_csv

In [4]:
columns = ['Date', 'Time','Bar OPEN Bid Quote','Bar HIGH Bid Quote','Bar LOW Bid Quote','Bar CLOSE Bid Quote','Volume']
base = '../data/DAT_MT_EURUSD_M1_20'
test_path = r'C:\\Users\\jarai\\Desktop\\Python_work\\TheBridge\\Alumno\\Javier_Araiz_TheBridge\\ML_Project\\data'
first_file = 'merged_file1'

In [5]:
merge_csv(base, start_number= 15, range_number= (1, 5), column_name= columns, path= test_path, file_name= first_file )

In [6]:
base2 = '../data/DAT_MT_EURUSD_M1_20210'
second_file = 'merged_file2'

In [7]:
merge_csv(base2, start_number= 0, range_number= (1, 5), column_name= columns, file_name= second_file, path= test_path )

In [8]:
base3 ='../data/merged_file'
final_name = 'all_joined_forex'

In [9]:
csv_one = '../data/merged_file1.csv'
csv_two = '../data/merged_file2.csv'
name_of_final = 'all_together_forex'

In [10]:
save_merged_csv(csv_one, csv_two,file_name= name_of_final, path= test_path)

In [13]:
df = pd.read_csv('../data/all_together_forex.csv')
df

,Date,Time,Bar OPEN Bid Quote,Bar HIGH Bid Quote,Bar LOW Bid Quote,Bar CLOSE Bid Quote,Volume
0,2016.01.03,17:00,1.08701,1.08713,1.08701,1.08713,0
1,2016.01.03,17:01,1.08712,1.08712,1.08712,1.08712,0
2,2016.01.03,17:02,1.08708,1.08722,1.08708,1.08722,0
3,2016.01.03,17:03,1.08717,1.08723,1.08717,1.08723,0
4,2016.01.03,17:04,1.08718,1.08718,1.08711,1.08711,0
...,...,...,...,...,...,...,...
2013228,2021.05.31,19:53,1.22332,1.22332,1.22329,1.22330,0
2013229,2021.05.31,19:54,1.22330,1.22330,1.22329,1.22329,0
2013230,2021.05.31,19:55,1.22327,1.22331,1.22326,1.22330,0
2013231,2021.05.31,19:57,1.22329,1.22329,1.22329,1.22329,0


In [16]:
df.isna().value_counts()

Date   Time   Bar OPEN Bid Quote  Bar HIGH Bid Quote  Bar LOW Bid Quote  Bar CLOSE Bid Quote  Volume
False  False  False               False               False              False                False     2013233
dtype: int64